In [3]:
import os
os.environ["JAVA_HOME"] = "C:\Java\jre1.8.0_333"
os.environ["SPARK_HOME"] = "C:\Spark\spark-3.2.1-bin-hadoop3.2"


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
dataset.head()

Row(_id=1, idx=0, vervar='ACEH', var='Angka Partisipasi Sekolah (APS)  ', tahun=2017, variable='SD', value='99.85')

In [8]:
dataset = spark.read.csv('C:/Users/dedededededededed/Downloads/LESSON/Materi Training Big Data/Modul_Day1/Lab/dataset_prep/sekolah.csv',inferSchema=True, header =True, sep=",")
dataset.printSchema()

root
 |-- _id: integer (nullable = true)
 |-- idx: integer (nullable = true)
 |-- vervar: string (nullable = true)
 |-- var: string (nullable = true)
 |-- tahun: integer (nullable = true)
 |-- variable: string (nullable = true)
 |-- value: string (nullable = true)



In [10]:
dataset.columns

['_id', 'idx', 'vervar', 'var', 'tahun', 'variable', 'value']

In [11]:
dataset =dataset.drop("_id","idx")
dataset.head()


Row(vervar='ACEH', var='Angka Partisipasi Sekolah (APS)  ', tahun=2017, variable='SD', value='99.85')

In [12]:
dataset = dataset.withColumnRenamed('vervar','propinsi')
dataset.columns


['propinsi', 'var', 'tahun', 'variable', 'value']

In [14]:
dataset.show(5)

+--------------+--------------------+-----+--------+-----+
|      propinsi|                 var|tahun|variable|value|
+--------------+--------------------+-----+--------+-----+
|          ACEH|Angka Partisipasi...| 2017|      SD|99.85|
|SUMATERA UTARA|Angka Partisipasi...| 2017|      SD|99.49|
|SUMATERA BARAT|Angka Partisipasi...| 2017|      SD| 99.5|
|          RIAU|Angka Partisipasi...| 2017|      SD|98.98|
|         JAMBI|Angka Partisipasi...| 2017|      SD|99.59|
+--------------+--------------------+-----+--------+-----+
only showing top 5 rows



In [15]:
print(dataset.show(5))
tmpds = dataset.filter("propinsi != 'ACEH' and propinsi != 'JAMBI'")
print(tmpds.show(5))

+--------------+--------------------+-----+--------+-----+
|      propinsi|                 var|tahun|variable|value|
+--------------+--------------------+-----+--------+-----+
|          ACEH|Angka Partisipasi...| 2017|      SD|99.85|
|SUMATERA UTARA|Angka Partisipasi...| 2017|      SD|99.49|
|SUMATERA BARAT|Angka Partisipasi...| 2017|      SD| 99.5|
|          RIAU|Angka Partisipasi...| 2017|      SD|98.98|
|         JAMBI|Angka Partisipasi...| 2017|      SD|99.59|
+--------------+--------------------+-----+--------+-----+
only showing top 5 rows

None
+----------------+--------------------+-----+--------+-----+
|        propinsi|                 var|tahun|variable|value|
+----------------+--------------------+-----+--------+-----+
|  SUMATERA UTARA|Angka Partisipasi...| 2017|      SD|99.49|
|  SUMATERA BARAT|Angka Partisipasi...| 2017|      SD| 99.5|
|            RIAU|Angka Partisipasi...| 2017|      SD|98.98|
|SUMATERA SELATAN|Angka Partisipasi...| 2017|      SD|99.63|
|        BEN

In [16]:
dataset.groupBy('propinsi','tahun','variable').count().filter('count > 1').show()

+--------+-----+--------+-----+
|propinsi|tahun|variable|count|
+--------+-----+--------+-----+
|   PAPUA| 2017|      PT|    2|
|   PAPUA| 2017|      SD|    2|
|   PAPUA| 2017|     SMP|    2|
|   PAPUA| 2017|     SMA|    2|
+--------+-----+--------+-----+



In [27]:
data = dataset.dropDuplicates(['propinsi','tahun','variable'])

In [29]:
data.groupBy('propinsi','tahun','variable').count().filter('count > 1').show()

+--------+-----+--------+-----+
|propinsi|tahun|variable|count|
+--------+-----+--------+-----+
+--------+-----+--------+-----+



In [30]:
data.where('value is null').show()

+-------------+--------------------+-----+--------+-----+
|     propinsi|                 var|tahun|variable|value|
+-------------+--------------------+-----+--------+-----+
|DI YOGYAKARTA|Angka Partisipasi...| 2017|      SD| null|
+-------------+--------------------+-----+--------+-----+



In [32]:
kelas = data.na.fill({'value':80})
kelas.show()

+-------------+--------------------+-----+--------+-----+
|     propinsi|                 var|tahun|variable|value|
+-------------+--------------------+-----+--------+-----+
|         ACEH|Angka Partisipasi...| 2017|      PT|34.28|
|         ACEH|Angka Partisipasi...| 2017|      SD|99.85|
|         ACEH|Angka Partisipasi...| 2017|     SMA|82.15|
|         ACEH|Angka Partisipasi...| 2017|     SMP|98.09|
|         BALI|Angka Partisipasi...| 2017|      PT|26.56|
|         BALI|Angka Partisipasi...| 2017|      SD|99.44|
|         BALI|Angka Partisipasi...| 2017|     SMA|82.16|
|         BALI|Angka Partisipasi...| 2017|     SMP|97.72|
|       BANTEN|Angka Partisipasi...| 2017|      PT|21.33|
|       BANTEN|Angka Partisipasi...| 2017|      SD|99.31|
|       BANTEN|Angka Partisipasi...| 2017|     SMA|67.77|
|       BANTEN|Angka Partisipasi...| 2017|     SMP|95.67|
|     BENGKULU|Angka Partisipasi...| 2017|      PT| 29.9|
|     BENGKULU|Angka Partisipasi...| 2017|      SD| 99.8|
|     BENGKULU

In [33]:
kelas.where('value is null').show()
kelas.where("propinsi = 'DI YOGYAKARTA'").show()

+--------+---+-----+--------+-----+
|propinsi|var|tahun|variable|value|
+--------+---+-----+--------+-----+
+--------+---+-----+--------+-----+

+-------------+--------------------+-----+--------+-----+
|     propinsi|                 var|tahun|variable|value|
+-------------+--------------------+-----+--------+-----+
|DI YOGYAKARTA|Angka Partisipasi...| 2017|      PT|51.33|
|DI YOGYAKARTA|Angka Partisipasi...| 2017|      SD|   80|
|DI YOGYAKARTA|Angka Partisipasi...| 2017|     SMA|87.61|
|DI YOGYAKARTA|Angka Partisipasi...| 2017|     SMP|99.63|
+-------------+--------------------+-----+--------+-----+



In [34]:
from pyspark.sql import functions as f
dsedit = dsnona.withColumn('value',f.when(dsnona.propinsi == 'DI YOGYAKARTA',100).otherwise(dsnona.value))
dsedit.where("propinsi = 'DI YOGYAKARTA'").show()

+-------------+--------------------+-----+--------+-----+
|     propinsi|                 var|tahun|variable|value|
+-------------+--------------------+-----+--------+-----+
|DI YOGYAKARTA|Angka Partisipasi...| 2017|      PT|  100|
|DI YOGYAKARTA|Angka Partisipasi...| 2017|      SD|  100|
|DI YOGYAKARTA|Angka Partisipasi...| 2017|     SMA|  100|
|DI YOGYAKARTA|Angka Partisipasi...| 2017|     SMP|  100|
+-------------+--------------------+-----+--------+-----+

